In [1]:
import os
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [3]:

# Initialize document processor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Load documents
loader = DirectoryLoader(
    "/Users/pherbert/Documents/GoHealth Projects/model-plan-recommendation/modelplanrecommendation",  # Or your specific path
    glob=["**/*.py", "**/*.txt", "**/*.md"],
    loader_cls=TextLoader,
)
docs = loader.load()
splits = text_splitter.split_documents(docs)

# Create vectorstore
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create retrieval chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
llm = ChatOllama(model="deepseek-r1:14b")

# Build basic QA chain
prompt = ChatPromptTemplate.from_template("""
Answer questions based on the provided context.

Context:
{local_docs}

Question: {question}
""")

chain = (
    RunnablePassthrough()
    | RunnablePassthrough.assign(
        local_docs=lambda x: retriever.invoke(x["question"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

# Debug what's in the vectorstore
result = vectorstore.get()
file_counts = {}
for metadata in result['metadatas']:
    source = metadata.get('source', 'Unknown')
    filename = os.path.basename(source)
    file_counts[filename] = file_counts.get(filename, 0) + 1

print("\nFiles in vectorstore:")
for filename, count in sorted(file_counts.items()):
    print(f"📄 {filename}: {count} chunks")



Files in vectorstore:
📄 analysis.py: 68 chunks
📄 base_model.py: 4 chunks
📄 benefit_map.py: 1 chunks
📄 config.py: 7 chunks
📄 constants_calculator.py: 40 chunks
📄 custom_warnings.py: 1 chunks
📄 data_ingestion.py: 19 chunks
📄 exceptions.py: 2 chunks
📄 external_data_processor.py: 49 chunks
📄 flask_responses.py: 9 chunks
📄 global_weights.py: 7 chunks
📄 heuristic_model_v1.py: 16 chunks
📄 heuristic_model_v2.py: 20 chunks
📄 invocations_schema.py: 15 chunks
📄 neural_network_model.py: 25 chunks
📄 node.py: 15 chunks
📄 optimization.py: 55 chunks
📄 payload_parser.py: 56 chunks
📄 plan_fit_graph.py: 55 chunks
📄 preprocessor.py: 88 chunks
📄 sample_model.py: 2 chunks
📄 trainable_heuristic_model.py: 31 chunks
📄 utilities.py: 12 chunks


In [4]:
# Test retrieval for a specific file
question = "summarize trainable_heuristic_model.py"
retrieved_docs = retriever.invoke(question)

print(f"\nRetrieved documents for query: '{question}'")
for i, doc in enumerate(retrieved_docs, 1):
    source = doc.metadata.get('source', 'Unknown')
    filename = os.path.basename(source)
    print(f"\n{i}. {filename}")
    print(f"Content preview: {doc.page_content[:200]}...")

# Try the actual query
response = chain.invoke({"question": question})
print("\nFull response from model:")
print(response)


Retrieved documents for query: 'summarize trainable_heuristic_model.py'

1. config.py
Content preview: # all available models to be selected from
model_dict = {
    "heuristic_v1": HeuristicModelOne(),
    "heuristic_v2": HeuristicModelTwo(GLOBAL_WEIGHTS_HEURISTIC_V2),
    "heuristic_v2.1": HeuristicMo...

2. heuristic_model_v2.py
Content preview: class HeuristicModelTwo(BaseModel):
    def __init__(
        self,
        global_weights: dict,
        model_version: str = "1.0",
        logging_level: int = logging.DEBUG,
        is_simulation:...

3. heuristic_model_v2.py
Content preview: @property
    def model_name(self) -> str:
        model_name = "heuristic_python"
        return model_name

    @property
    def model_version(self) -> str:
        return self._model_version

    ...

4. config.py
Content preview: ),
    "heuristic_v2.6": HeuristicModelTwo(
        global_weights=GLOBAL_WEIGHTS_HEURISTIC_V2_6, model_version="1.6"
    ),
    "heuristic_v2.7": HeuristicModelTwo(
 

In [5]:
# Get documents specifically from trainable_heuristic_model.py
result = vectorstore.get()
relevant_chunks = []

for i, metadata in enumerate(result['metadatas']):
    if os.path.basename(metadata.get('source', '')) == 'trainable_heuristic_model.py':
        relevant_chunks.append({
            'content': result['documents'][i],
            'metadata': metadata
        })

print(f"\nFound {len(relevant_chunks)} chunks from trainable_heuristic_model.py:")
for i, chunk in enumerate(relevant_chunks[:3], 1):  # Show first 3 chunks
    print(f"\nChunk {i}:")
    print(chunk['content'][:300], "...")


Found 31 chunks from trainable_heuristic_model.py:

Chunk 1:
import logging

import newrelic.agent
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from modelplanrecommendation.businesslogic.external_data_processor import (
    append_external_data,
    get_ext_data_scores,
)
from modelplanrecommendati ...

Chunk 2:
MODEL_VERSIONS_TO_APPEND_EXT_DATA = [
    "2.2.0",
    "2.3.0",
    "2.4.0",
    "2.5.0",
    "2.6.0",
    "2.7.0",
    "2.8.0",
    "3.0.0",
    "3.1.0",
    "3.2.0",
    "4.0.0", # Calling this 4.0 for now
]


class TrainableNode(nn.Module):
    def __init__(
        self,
        name: str,
      ...

Chunk 3:
Parameters
        ----------
        name : str
            The name of the node.
        input_nodes : list of TrainableNode, optional
            List of input nodes connected to this node. Defaults to None.
        initial_weights : list of float, optional
            List of initial weights to  ...


In [6]:
import multiprocessing